In [1]:
%load_ext autoreload
%autoreload 2

In [18]:
from src import parser
import numpy as np
from simplexnoise.noise import SimplexNoise, normalize
from PIL import Image

In [134]:
s3d = SimplexNoise(num_octaves=7, persistence=0.1, dimensions=3)
s2d = SimplexNoise(num_octaves=7, persistence=0.1, dimensions=2)

In [123]:
def noise(x_size, y_size, z_size):
    pass

def edge_distance(x, y, x_size, y_size):
    x0 = x_size - x
    y0 = y_size - y
    return min(x,y,x0,y0)

def edge_contrib(x, y, x_size, y_size, d_threshold):
    d = edge_distance(x,y,x_size,y_size)
    if d < d_threshold:
        d_contrib = d/d_threshold
    else:
        d_contrib = 1
    return d_contrib

def circle_contrib(x, y, x_size, y_size, d_threshold):
    center = np.array([x_size/2, y_size/2])
    xy = np.array([x,y])
    cv = xy - center
    #cd = (cv @ cv)**(1/2)
    cd = (cv @ cv)**(3/4)
    if cd == 0:
        return 1
    else:
        val = d_threshold * 1/cd
        if val > 1:
            return 1
        else:
            return val

In [124]:
def y_noise(x_size, y_size, d_threshold):
    a = np.zeros((x_size, y_size), np.uint8)
    for index, _ in np.ndenumerate(a):
        x,y = index
        xy = np.array([x,y])
        contrib = circle_contrib(x,y,x_size,y_size,d_threshold)
        a[index] = contrib * normalize(s2d.fractal(x, y, hgrid=x_size)) * 255
        #a[index] = normalize(s2d.fractal(x, y, hgrid=x_size)) * 255
    return a

In [135]:
a = y_noise(100, 100, 90)

In [136]:
img = Image.fromarray(a, mode="L")

In [137]:
img.save("ayy.png")